# Week 1 Lab: next generation sequencing and mutation hunting
## Prelab 2
**Due: Thursday 04/04/19 10am**

Today's prelab will focus on understanding **sequencing coverage** and how to model coverage in the presence of mutations.

## 1. Exploring sequencing coverage

## 2. Reviewing the binomial distribution

TODO include p-value distribution